Question 1: How would you describe TensorFlow in a short sentence? What are its main features? Can you name other popular Deep Learning libraries?

According to Tensorflow Documentation:

TensorFlow is a free and open-source software library for machine learning and artificial intelligence. It can be used across a range of tasks but has a particular focus on training and inference of deep neural networks

Other Deep Learning Libraries:

Pytorch

Theano

Keras

Deeplearning4j
   

Question 2:Is TensorFlow a drop-in replacement for NumPy? What are the main differences between the two?

NumPy was developed as a full-fledged open source tensor algebra package for Python that could rival MATLAB and the likes. It is a Python library with a long history and plenty of functionality, either directly in it or built around it (see SciPy and different scikits). TensorFlow was developed by Google much more recently specifically for the purpose of building machine learning models (although you could use it for many other tasks), continuing the ideas from the (now discontinued) Theano library. Although TensorFlow is most commonly used with Python, it can be used in C/C++ and other languages too, which is important because it allows you to train a model in Python and then integrate it in an existing application written in another language.

A main selling point of TensorFlow is that it can automatically differentiate computations. This is an essential feature for deep learning, that uses gradient-based optimization (backpropagation), and it means that you can pretty much just write whatever you want to compute and TensorFlow will figure out the gradients by itself. There are things like Autograd or JAX for NumPy, but they are not as powerful as TensorFlow automatic differentiation, which actually maintains a computation graph structure under the hood (the name "TensorFlow" refers to the tensors and their gradients "flowing" through the computation graph).

TensorFlow offers a bunch of useful stuff if you are doing machine learning, like training checkpoints, distributed training, TensorBoard, TensorFlow Serving, etc. It also integrates better (or at all) with inference platforms and standards like TensorRT, Google Coral, ONNX and that kind of stuff.

TensorFlow integrates a lot more functionality that is not strictly array manipulation into the library itself, like image manipulation and common neural network utilities. NumPy tends to defer that kind of things to additional libraries like SciPy, making it more of an ecosystem and less monolithic. TensorFlow has some of that too, like TensorFlow Probability or TensorFlow Graphics, but it is not too developed yet.

Question 3: Do you get the same result with tf.range(10) and tf.constant(np.arange(10))?

In [1]:
import tensorflow as tf
print(tf.range(10))

tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)


In [3]:
import tensorflow as tf
import numpy as np
print(tf.constant(np.arange(10)))

tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)


tf.constant returns A Constant Tensor.

tf.range returns An 1-D Tensor of type dtype.

Question 4:Can you name six other data structures available in TensorFlow, beyond regular tensors?

Different Datastructures are:

Constant

Variable

Place holder

A tensor with nth shape

dual vectors

Scalars

Question 5:A custom loss function can be defined by writing a function or by subclassing the keras.losses.Loss class. When would you use each option?

In the second loss function the reduction parameter controls the way the output is aggregated, eg. taking the sum of elements or summing over the batch etc. By default, your code uses keras.losses.Reduction.AUTO, which translates into summing over the batch if you check the source code. This means, the final loss will be a vector, but there are other reductions available, you can check them in the docs. I believe even if you do not define the reduction to take the sum of the loss elements in the loss vector, TF optimizers will do so, to avoid errors from backpropagating a vector. Backpropagation on a vector would cause problems at weights that "contribute" to every loss element.

Question 6:Similarly, a custom metric can be defined in a function or a subclass of keras.metrics.Metric. When would you use each option?

In the second metric function the reduction parameter controls the way the output is aggregated, eg. taking the sum of elements or summing over the batch etc. By default, your code uses keras.metrics.Reduction.AUTO, which translates into summing over the batch if you check the source code. This means, the final metric will be a vector, but there are other reductions available, you can check them in the docs. I believe even if you do not define the reduction to take the sum of the metric elements in the metric vector, TF optimizers will do so, to avoid errors from backpropagating a vector. Backpropagation on a vector would cause problems at weights that "contribute" to every metric element

Question 7: When should you create a custom layer versus a custom model?`

The Model class has the same API as Layer, with the following differences: - It exposes built-in training, evaluation, and prediction loops (model.fit(), model.evaluate(), model.predict()). - It exposes the list of its inner layers, via the model.layers property. - It exposes saving and serialization APIs.

Effectively, the "Layer" class corresponds to what we refer to in the literature as a "layer" (as in "convolution layer" or "recurrent layer") or as a "block" (as in "ResNet block" or "Inception block").

Meanwhile, the "Model" class corresponds to what is referred to in the literature as a "model" (as in "deep learning model") or as a "network" (as in "deep neural network").

Question 8:What are some use cases that require writing your own custom training loop?

We can create custom training loop when we want very low level control of our code execution.

Question 9: Can custom Keras components contain arbitrary Python code, or must they be convertible to TF Functions?

We can tell Keras not to convert your Python functions to TF
Functions by setting dynamic=True when creating a custom layer
or a custom model. Alternatively, you can set run_eagerly=True
when calling the model’s compile() method.

Question 10: What are the main rules to respect if you want a function to be convertible to a TF Function?

If you call any external library, including NumPy or even the standard library,
this call will run only during tracing, it will not be part of the graph. Indeed, a
TensorFlow graph can only include TensorFlow constructs (tensors, operations,
variables, datasets, and so on). So make sure you use tf.reduce_sum() instead of
np.sum(), and tf.sort() instead of the built-in sorted() function, and so on
(unless you really want the code to run only during tracing).

• You can call other Python functions or TF Functions, but they should follow the
same rules, as TensorFlow will also capture their operations in the computation
graph. Note that these other functions do not need to be decorated with
@tf.function.

If the function creates a TensorFlow variable (or any other stateful TensorFlow
object, such as a dataset or a queue), it must do so upon the very first call, and
only then, or else you will get an exception. It is usually preferable to create vari‐
ables outside of the TF Function (e.g., in the build() method of a custom layer).

The source code of your Python function should be available to TensorFlow. If
the source code is unavailable (for example, if you define your function in the
Python shell, which does not give access to the source code, or if you deploy only
the compiled Python files *.pyc to production), then the graph generation pro‐
cess will fail or have limited functionality.
• TensorFlow will only capture for loops that iterate over a tensor or a Dataset. So
make sure you use for i in tf.range(10) rather than for i in range(10), or
else the loop will not be captured in the graph. Instead, it will run during tracing.
This may be what you want, if the for loop is meant to build the graph, for exam‐
ple to create each layer in a neural network.
• And as always, for performance reasons, you should prefer a vectorized imple‐
mentation whenever you can, rather than using loops.